## Import

In [1]:
# notebooky stuff
%load_ext autoreload
%autoreload 2
from IPython.display import display

import sys 
sys.path.append('../../modules')

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import eumf_data, eumf_eval, eumf_pipeline
from sklearn import preprocessing, linear_model, model_selection, ensemble, feature_selection
import seaborn as sns
import itertools

# pandas pretty output
pd.set_option('display.min_rows', 20)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 3)

## Data

### Load all data

In [2]:
# migration rates
df_values = eumf_data.load_registrations_from_csv(impute_missing=True)

# google trends
df_trends = eumf_data.load_trends_from_csv()

countries = eumf_data.get_countries()
keyword_ids = df_trends.columns.levels[0].tolist()

df_gdp = eumf_data.read_gdp()
df_unempl = eumf_data.read_unempl()


### Join data, combine countries

In [3]:
# list of countries
countries = eumf_data.get_countries()
countries.remove("CY")

# migration rates
df_values = eumf_data.load_registrations_from_csv(impute_missing=True, countries=countries)

# google trends
df_trends = eumf_data.load_trends_from_csv(countries=countries)
keyword_ids = df_trends.columns.levels[0].tolist()

# macroeconomic data
df_gdp = eumf_data.read_gdp(countries=countries)
df_unempl = eumf_data.read_unempl(countries=countries)

country_combinations = [
    # ["GR", "CY"],
    ["LV", "LT", "EE"],
    ["BE", "NL", "LU"],
    ["CZ", "SK"],
    ["SE", "FI", "DK"],
    ["AT", "CH"]
]

panel = df_values.join(df_trends, how="outer")
panel_3m = panel.resample("3M", closed="left").mean()

panel_comb = eumf_data.combine_countries(panel, combinations=country_combinations)
panel_comb_3m = eumf_data.combine_countries(panel_3m, combinations=country_combinations)
df_gdp_comb = eumf_data.combine_countries(df_gdp, combinations=country_combinations)
df_unempl_comb = eumf_data.combine_countries(
    df_unempl, combinations=country_combinations, average=True
)
# note: strictly, unweighted average is wrong for unemployment, but should work in most cases

panel_comb_3m_macro = panel_comb_3m.join(df_gdp_comb).join(df_unempl_comb)

panel_comb_3m_macro["2017":"2018"]



10                                                                                                                       11                                                                                                                                            112                                                                                                               ...   value                                                                                                                                     gdp                                                                                                                                                unempl                                                                                                      
country     AT+CH  BE+NL+LU  BG  CZ+SK    ES      FR      GB    GR   HR   HU   IE     IT   LV+LT+EE    PL    PT   RO  SE+FI+DK  SI   AT+CH  BE+NL+LU    BG    CZ+SK     ES      FR      GB      GR     HR      HU      IE      IT   LV+LT+EE    PL      PT      RO   SE+FI+DK  SI   AT+CH  BE+NL+LU    BG    CZ+SK      ES      FR      GB      GR      HR      HU      IE      IT   LV+LT+EE    PL    ...     ES        FR        GB        GR        HR        HU       IE        IT     LV+LT+EE     PL        PT        RO     SE+FI+DK    SI     AT+CH   BE+NL+LU    BG    CZ+SK     ES      FR      GB      GR      HR      HU      IE       IT   LV+LT+EE    PL      PT      RO   SE+FI+DK    SI   AT+CH  BE+NL+LU  BG  CZ+SK   ES   FR   GB    GR    HR   HU   IE    IT  LV+LT+EE  PL   PT   RO  SE+FI+DK  SI 
date                                                                                                                                                                                                                                                                                                                                                                                                   ...                                                                                                                                                                                                                                                                                                                                                                                                            
2017-03-31  10.524  16.048   0.0   0.0  26.667  21.857  21.381  0.0  0.0  0.0  0.0  14.095    0.0     6.095  0.0  0.0  21.286   0.0  34.714  60.190    2.190  10.857  38.714  29.810  53.381  10.000  0.000   0.619  11.095  22.048    0.0    11.952  12.714  11.571  26.524   0.0  97.190  178.952  42.238  127.048  33.143  63.333  47.286   9.000  63.095  62.810  59.333  14.905  179.429  39.524  ...  1689.000  1268.333  1080.333  2327.333  4459.000  3747.667  156.000  5255.333  1396.333  12264.667  866.333  18186.333  654.333  420.333  28780.0  42970.0  1510.0  7620.0  5950.0  8450.0  9080.0  3760.0  2590.0  2820.0  14640.0  6800.0  10490.0  2760.0  4520.0  1850.0  33780.0  4790.0  2.85    6.433   6.6  6.00  18.2  9.6  4.6  22.4  12.7  4.3  7.4  11.6   7.333   5.3  9.8  5.3   7.167   7.4
2017-06-30   4.952  14.810   0.0   0.0  21.048  13.952  18.857  0.0  0.0  0.0  0.0  15.286    0.0     6.524  0.0  0.0  27.714   0.0  39.286  61.619   10.619   7.714  24.381  27.952  55.714   4.714  2.333   7.095  20.619  17.286    0.0    13.810  23.143  11.143  32.857   0.0  85.619  168.905  35.190   83.429  23.810  57.857  48.000  11.905  33.048  27.905  62.810  14.286  126.667  21.476  ...  1427.667  1023.333  1049.000  2113.333  4121.333  3637.667  164.667  4699.000  1407.000  12940.000  609.667  19264.000  520.333  365.000  29250.0  44540.0  1800.0  8470.0  6330.0  8560.0  8980.0  4120.0  2990.0  3200.0  14700.0  7140.0  11780.0  2960.0  4790.0  2200.0  35590.0  5260.0  2.70    5.867   6.2  5.75  17.3  9.6  4.4  21.8  11.5  4.3  6.7  11.3   7.667   5.1  9.2  4.9   7.133   6.6
2017-09-30   4.905  16.095   0.0   0.0  12.429  21.048  16.

## Experiments

In [4]:
T_TEST_MIN = "2014"
T_TEST_MAX = "2014"



### Which AR order?

up to 8

In [16]:
### TRAINING
t_min = "2010"
t_max = "2019"

n_years_eff = int(t_max) - int(t_min) - 1

cv_default = model_selection.KFold(n_splits=n_years_eff, shuffle=False)

feature_combinations = [["value"], ["value", "19"], ["19"]]

lag_order = [1, 2, 3, 4, 5, 6, 7, 8]

tuners = []
cv_scores, test_scores = [], []
train_stackeds, test_stackeds, train_unstackeds = [], [], []
model_names = []

# params = {}
params = {
    "randomforestregressor__max_features": ["auto", "sqrt"],
    "randomforestregressor__min_samples_leaf": [1, 2, 4, 8],
    "randomforestregressor__min_samples_split": [2, 4, 8, 16],
}

for features, order in itertools.product(feature_combinations, lag_order):

    model_names.append("+".join(features) + "_" + str(order))

    labeled = eumf_pipeline.prepare_data(
        panel_comb_3m_macro,
        columns=features,
        lags=list(range(1, order + 1)),
        t_min=t_min,
        t_max=t_max,
    )
    transformed = eumf_pipeline.transform_data(labeled)
    train, test = eumf_pipeline.split_data(
        transformed, t_test_min=T_TEST_MIN, t_test_max=T_TEST_MAX
    )
    train_stacked, test_stacked = eumf_pipeline.stack_data(train, test)

    train_stackeds.append(train_stacked)
    train_unstackeds.append(train)
    test_stackeds.append(test_stacked)

    tuner = eumf_pipeline.train_reg_model(
        train_stacked,
        reg=ensemble.RandomForestRegressor(random_state=42),
        extra_pipeline_steps=[preprocessing.StandardScaler()],
        params=params,
        scoring=eumf_eval.scorer_rmse,
    )
    tuners.append(tuner)

    cv_score = eumf_eval.score_cv(tuner.best_estimator_, train_stacked, cv=cv_default,)
    cv_scores.append(cv_score)

    test_score = eumf_eval.score_test(tuner.best_estimator_, test_stacked,)
    test_scores.append(test_score)


In [17]:
pd.DataFrame({k: t.best_params_ for k, t in zip(model_names, tuners)}).transpose()

,randomforestregressor__max_features,randomforestregressor__min_samples_leaf,randomforestregressor__min_samples_split
value_1,auto,8,2
value_2,auto,8,2
value_3,auto,8,2
value_4,auto,4,2
value_5,auto,4,2
value_6,auto,4,2
value_7,auto,4,2
value_8,auto,4,2
value+19_1,auto,8,2
value+19_2,auto,8,2


In [18]:
eumf_eval.agg_multiple_cv_scores(cv_scores, model_names)


fit_time               score_time               test_mae               test_rmse               test_explained_variance               test_r2_mod               test_delta_mae              
             mean    std    sem      mean     std    sem     mean    std    sem      mean    std    sem             mean           std    sem       mean     std    sem        mean       std    sem  
value_1      0.326   0.093  0.033    0.028    0.007  0.003  -0.071   0.018  0.006   -0.102   0.032  0.011         3.607e-01        0.193  0.068     0.449    0.268  0.095      0.047      0.056  0.020
value_2      0.334   0.059  0.021    0.028    0.006  0.002  -0.070   0.019  0.007   -0.102   0.033  0.012         3.758e-01        0.172  0.061     0.462    0.241  0.085      0.047      0.057  0.020
value_3      0.310   0.059  0.021    0.025    0.007  0.002  -0.071   0.018  0.006   -0.103   0.033  0.012         3.747e-01        0.174  0.062     0.455    0.244  0.086      0.047      0.057  0.020
value_4      0.366   0.085  0.030    0.023    0.011  0.004  -0.069   0.015  0.005   -0.099   0.029  0.010         3.952e-01        0.178  0.063     0.471    0.248  0.088      0.049      0.061  0.022
value_5      0.329   0.038  0.013    0.023    0.009  0.003  -0.068   0.016  0.005   -0.098   0.030  0.011         4.095e-01        0.168  0.059     0.490    0.230  0.081      0.050      0.060  0.021
value_6      0.372   0.048  0.017    0.021    0.008  0.003  -0.068   0.015  0.005   -0.099   0.031  0.011         4.119e-01        0.161  0.057     0.490    0.227  0.080      0.050      0.060  0.021
value_7      0.322   0.086  0.030    0.020    0.008  0.003  -0.067   0.015  0.005   -0.098   0.030  0.011         4.124e-01        0.165  0.058     0.488    0.232  0.082      0.050      0.061  0.021
value_8      0.394   0.084  0.030    0.021    0.005  0.002  -0.067   0.016  0.006   -0.098   0.031  0.011         4.144e-01        0.168  0.059     0.491    0.232  0.082      0.050      0.060  0.021
value+19_1   0.250   0.050  0.018    0.022    0.005  0.002  -0.068   0.014  0.005   -0.099   0.029  0.010         3.765e-01        0.197  0.070     0.465    0.267  0.094      0.050      0.061  0.022
value+19_2   0.451   0.090  0.032    0.036    0.011  0.004  -0.066   0.015  0.005   -0.098   0.031  0.011         4.034e-01        0.162  0.057     0.491    0.231  0.082      0.052      0.060  0.021
value+19_3   0.319   0.035  0.012    0.019    0.005  0.002  -0.067   0.015  0.005   -0.099   0.031  0.011         4.062e-01        0.152  0.054     0.493    0.223  0.079      0.051      0.060  0.021
value+19_4   0.509   0.142  0.050    0.028    0.007  0.002  -0.066   0.014  0.005   -0.096   0.030  0.011         4.290e-01        0.149  0.053     0.512    0.219  0.078      0.052      0.062  0.022
value+19_5   0.381   0.047  0.017    0.018    0.004  0.001  -0.065   0.014  0.005   -0.095   0.030  0.011         4.367e-01        0.147  0.052     0.522    0.214  0.076      0.053      0.061  0.022
value+19_6   0.764   0.155  0.055    0.034    0.011  0.004  -0.065   0.014  0.005   -0.095   0.030  0.011         4.387e-01        0.145  0.051     0.523    0.212  0.075      0.053      0.061  0.022
value+19_7   0.725   0.168  0.060    0.029    0.012  0.004  -0.064   0.013  0.005   -0.095   0.029  0.010         4.404e-01        0.151  0.053     0.524    0.216  0.076      0.053      0.062  0.022
value+19_8   0.670   0.142  0.050    0.027    0.009  0.003  -0.065   0.014  0.005   -0.095   0.030  0.011         4.368e-01        0.152  0.054     0.522    0.217  0.077      0.053      0.062  0.022
19_1         0.158   0.019  0.007    0.016    0.004  0.001  -0.101   0.027  0.010   -0.143   0.041  0.014        -1.568e-01        0.485  0.172    -0.149    0.659  0.233      0.017      0.051  0.018
19_2         0.178   0.029  0.010    0.022    0.009  0.003  -0.093   0.029  0.010   -0.135   0.048  0.017         4.952e-02        0.290  0.102     0.058    0.459  0.162      0.025      0.050  0.018
19_3         0.229   0.038  0.0

up to 4, one more year in training

In [19]:
### TRAINING
t_min = "2009"
t_max = "2019"

n_years_eff = int(t_max) - int(t_min) - 1

cv_default = model_selection.KFold(n_splits=n_years_eff, shuffle=False)

feature_combinations = [["value"], ["value", "19"], ["19"]]

lag_order = [1, 2, 3, 4]

tuners = []
cv_scores, test_scores = [], []
train_stackeds, test_stackeds, train_unstackeds = [], [], []
model_names = []

# params = {}
params = {
    "randomforestregressor__max_features": ["auto", "sqrt"],
    "randomforestregressor__min_samples_leaf": [1, 2, 4, 8],
    "randomforestregressor__min_samples_split": [2, 4, 8, 16],
}

for features, order in itertools.product(feature_combinations, lag_order):

    model_names.append("+".join(features) + "_" + str(order))

    labeled = eumf_pipeline.prepare_data(
        panel_comb_3m_macro,
        columns=features,
        lags=list(range(1, order + 1)),
        t_min=t_min,
        t_max=t_max,
    )
    transformed = eumf_pipeline.transform_data(labeled)
    train, test = eumf_pipeline.split_data(
        transformed, t_test_min=T_TEST_MIN, t_test_max=T_TEST_MAX
    )
    train_stacked, test_stacked = eumf_pipeline.stack_data(train, test)

    train_stackeds.append(train_stacked)
    train_unstackeds.append(train)
    test_stackeds.append(test_stacked)

    tuner = eumf_pipeline.train_reg_model(
        train_stacked,
        reg=ensemble.RandomForestRegressor(random_state=42),
        extra_pipeline_steps=[preprocessing.StandardScaler()],
        params=params,
        scoring=eumf_eval.scorer_rmse,
    )
    tuners.append(tuner)

    cv_score = eumf_eval.score_cv(tuner.best_estimator_, train_stacked, cv=cv_default,)
    cv_scores.append(cv_score)

    test_score = eumf_eval.score_test(tuner.best_estimator_, test_stacked,)
    test_scores.append(test_score)


In [20]:
pd.DataFrame({k: t.best_params_ for k, t in zip(model_names, tuners)}).transpose()

,randomforestregressor__max_features,randomforestregressor__min_samples_leaf,randomforestregressor__min_samples_split
value_1,auto,4,16
value_2,auto,4,16
value_3,auto,4,16
value_4,auto,4,16
value+19_1,auto,2,16
value+19_2,auto,2,16
value+19_3,auto,4,16
value+19_4,auto,4,16
19_1,auto,8,2
19_2,auto,8,2


In [21]:
eumf_eval.agg_multiple_cv_scores([df[1:] for df in cv_scores], model_names)


fit_time               score_time               test_mae               test_rmse               test_explained_variance               test_r2_mod               test_delta_mae              
             mean    std    sem      mean     std    sem     mean    std    sem      mean    std    sem             mean           std    sem       mean     std    sem        mean       std    sem  
value_1      0.455   0.157  0.055    0.042    0.020  0.007  -0.069   0.014  0.005   -0.101   0.027  0.010           0.355          0.238  0.084     0.432    0.312  0.110      0.049      0.061  0.022
value_2      0.224   0.038  0.013    0.019    0.004  0.002  -0.068   0.014  0.005   -0.099   0.028  0.010           0.383          0.210  0.074     0.457    0.282  0.100      0.050      0.062  0.022
value_3      0.466   0.101  0.036    0.044    0.017  0.006  -0.069   0.014  0.005   -0.100   0.028  0.010           0.379          0.210  0.074     0.450    0.282  0.100      0.049      0.062  0.022
value_4      0.380   0.228  0.081    0.030    0.023  0.008  -0.066   0.013  0.005   -0.097   0.028  0.010           0.410          0.182  0.064     0.478    0.258  0.091      0.052      0.064  0.023
value+19_1   0.604   0.171  0.060    0.049    0.019  0.007  -0.066   0.011  0.004   -0.098   0.024  0.009           0.361          0.259  0.091     0.436    0.320  0.113      0.051      0.066  0.023
value+19_2   0.416   0.050  0.018    0.028    0.007  0.002  -0.065   0.011  0.004   -0.097   0.025  0.009           0.386          0.236  0.084     0.458    0.293  0.104      0.053      0.066  0.023
value+19_3   0.320   0.040  0.014    0.020    0.005  0.002  -0.066   0.012  0.004   -0.096   0.026  0.009           0.407          0.204  0.072     0.482    0.265  0.094      0.052      0.066  0.023
value+19_4   0.410   0.100  0.035    0.021    0.006  0.002  -0.065   0.012  0.004   -0.094   0.027  0.010           0.432          0.177  0.062     0.510    0.237  0.084      0.052      0.064  0.023
19_1         0.263   0.080  0.028    0.020    0.006  0.002  -0.105   0.031  0.011   -0.145   0.042  0.015          -0.125          0.408  0.144    -0.162    0.609  0.215      0.012      0.046  0.016
19_2         0.216   0.053  0.019    0.022    0.013  0.005  -0.096   0.026  0.009   -0.138   0.043  0.015          -0.047          0.372  0.132    -0.032    0.525  0.186      0.022      0.051  0.018
19_3         0.290   0.074  0.026    0.027    0.006  0.002  -0.092   0.032  0.011   -0.136   0.050  0.018           0.024          0.345  0.122     0.044    0.501  0.177      0.026      0.047  0.017
19_4         0.193   0.039  0.014    0.020    0.005  0.002  -0.091   0.033  0.012   -0.133   0.050  0.018           0.059          0.297  0.105     0.108    0.432  0.153      0.027      0.045  0.016

### Add absolute values to GT

In [22]:
### TRAINING

t_min = "2010"
t_max = "2019"
n_years_eff = int(t_max) - int(t_min) - 1
cv_default = model_selection.KFold(n_splits=n_years_eff, shuffle=False)

feature_combinations = [["value", "19"], ["19"], ["19", "gdp", "unempl"]]
w_absolutes = [False, True]

tuners = []
cv_scores, test_scores = [], []
train_stackeds, test_stackeds, train_unstackeds = [], [], []
model_names = []

# params = {}
params = {
    "randomforestregressor__max_features": ["auto", "sqrt"],
    "randomforestregressor__min_samples_leaf": [1, 2, 4, 8],
    "randomforestregressor__min_samples_split": [2, 4, 8, 16],
}

for features, w_absolutes in itertools.product(feature_combinations, w_absolutes):

    model_names.append("+".join(features) + "_" + str(w_absolutes))

    labeled = eumf_pipeline.prepare_data(
        panel_comb_3m_macro, columns=features, lags=list(range(1, 6)), t_min="2010"
    )
    transformed = eumf_pipeline.transform_data(labeled)
    if w_absolutes:
        extra_cols = [c for c in labeled.x.columns if c[0].startswith("19")]
        transformed.x = transformed.x.join(labeled.x[extra_cols], rsuffix="_abs")
    train, test = eumf_pipeline.split_data(
        transformed, t_test_min=T_TEST_MIN, t_test_max=T_TEST_MAX
    )
    train_stacked, test_stacked = eumf_pipeline.stack_data(train, test)

    train_stackeds.append(train_stacked)
    train_unstackeds.append(train)
    test_stackeds.append(test_stacked)

    tuner = eumf_pipeline.train_reg_model(
        train_stacked,
        reg=ensemble.RandomForestRegressor(random_state=42),
        extra_pipeline_steps=[preprocessing.StandardScaler()],
        params=params,
        scoring=eumf_eval.scorer_rmse,
    )
    tuners.append(tuner)

    cv_score = eumf_eval.score_cv(tuner.best_estimator_, train_stacked, cv=cv_default,)
    cv_scores.append(cv_score)

    test_score = eumf_eval.score_test(tuner.best_estimator_, test_stacked,)
    test_scores.append(test_score)


In [23]:
pd.DataFrame({k: t.best_params_ for k, t in zip(model_names, tuners)}).transpose()

,randomforestregressor__max_features,randomforestregressor__min_samples_leaf,randomforestregressor__min_samples_split
value+19_False,auto,4,2
value+19_True,auto,8,2
19_False,auto,1,16
19_True,sqrt,1,2
19+gdp+unempl_False,sqrt,1,2
19+gdp+unempl_True,sqrt,4,2


In [24]:
eumf_eval.agg_multiple_cv_scores(cv_scores, model_names)


fit_time               score_time                   test_mae               test_rmse               test_explained_variance               test_r2_mod               test_delta_mae              
                      mean    std    sem      mean     std      sem       mean    std    sem      mean    std    sem             mean           std    sem       mean     std    sem        mean       std    sem  
value+19_False        0.380   0.064  0.023    0.018    0.008  2.699e-03  -0.065   0.014  0.005   -0.095   0.030  0.011           0.437          0.147  0.052     0.522    0.214  0.076      0.053      0.061  0.022
value+19_True         0.476   0.145  0.051    0.020    0.007  2.453e-03  -0.065   0.015  0.005   -0.097   0.032  0.011           0.434          0.140  0.050     0.520    0.209  0.074      0.052      0.060  0.021
19_False              0.358   0.072  0.025    0.020    0.007  2.512e-03  -0.086   0.026  0.009   -0.129   0.043  0.015          -0.005          0.404  0.143     0.113    0.505  0.179      0.031      0.049  0.017
19_True               0.270   0.062  0.022    0.023    0.009  3.178e-03  -0.087   0.030  0.011   -0.127   0.049  0.017           0.110          0.276  0.097     0.188    0.391  0.138      0.031      0.047  0.017
19+gdp+unempl_False   0.150   0.020  0.007    0.012    0.001  4.650e-04  -0.083   0.027  0.010   -0.118   0.044  0.016           0.202          0.180  0.064     0.311    0.282  0.100      0.035      0.049  0.017
19+gdp+unempl_True    0.135   0.025  0.009    0.011    0.002  6.386e-04  -0.085   0.027  0.010   -0.118   0.042  0.015           0.185          0.192  0.068     0.295    0.302  0.107      0.033      0.049  0.017

### Normalization scheme of x values

In [9]:
### TRAINING

t_min = "2010"
t_max = "2019"
n_years_eff = int(t_max) - int(t_min) - 1
cv_default = model_selection.KFold(n_splits=n_years_eff, shuffle=False)

feature_combinations = [["19"], ["19", "gdp", "unempl"], ["19", "gdp", "unempl", "value"]]
epsilons = [0.1, 1., 5., 10.]

tuners = []
cv_scores, test_scores = [], []
train_stackeds, test_stackeds, train_unstackeds = [], [], []
model_names = []

# params = {}
params = {
}

for features, eps in itertools.product(feature_combinations, epsilons):

    model_names.append("+".join(features) + "_" + str(eps))

    labeled = eumf_pipeline.prepare_data(
        panel_comb_3m_macro, columns=features, lags=list(range(1, 6)), t_min="2010"
    )
    transformed = eumf_pipeline.transform_data(labeled, eps_x=eps)
    train, test = eumf_pipeline.split_data(
        transformed, t_test_min=T_TEST_MIN, t_test_max=T_TEST_MAX
    )
    train_stacked, test_stacked = eumf_pipeline.stack_data(train, test)

    train_stackeds.append(train_stacked)
    train_unstackeds.append(train)
    test_stackeds.append(test_stacked)

    tuner = eumf_pipeline.train_reg_model(
        train_stacked,
        reg=ensemble.RandomForestRegressor(random_state=42),
        extra_pipeline_steps=[preprocessing.StandardScaler()],
        params=params,
        scoring=eumf_eval.scorer_rmse,
    )
    tuners.append(tuner)

    cv_score = eumf_eval.score_cv(tuner.best_estimator_, train_stacked, cv=cv_default,)
    cv_scores.append(cv_score)

    test_score = eumf_eval.score_test(tuner.best_estimator_, test_stacked,)
    test_scores.append(test_score)


In [10]:
eumf_eval.agg_multiple_cv_scores(cv_scores, model_names)


fit_time               score_time                       test_mae               test_rmse               test_explained_variance               test_r2_mod               test_delta_mae              
                           mean    std    sem      mean       std        sem       mean    std    sem      mean    std    sem             mean           std    sem       mean     std    sem        mean       std    sem  
19_0.1                     0.242   0.015  0.005    0.012    1.572e-03  5.558e-04  -0.088   0.026  0.009   -0.129   0.041  0.015          -0.017          0.388  0.137     0.105    0.497  0.176      0.030      0.049  0.017
19_1.0                     0.280   0.020  0.007    0.013    7.809e-04  2.761e-04  -0.088   0.026  0.009   -0.130   0.041  0.014          -0.031          0.371  0.131     0.089    0.487  0.172      0.030      0.049  0.017
19_5.0                     0.400   0.145  0.051    0.021    1.061e-02  3.750e-03  -0.090   0.028  0.010   -0.134   0.044  0.016          -0.039          0.225  0.080     0.093    0.364  0.129      0.028      0.048  0.017
19_10.0                    0.488   0.176  0.062    0.025    1.079e-02  3.817e-03  -0.089   0.026  0.009   -0.135   0.045  0.016          -0.068          0.313  0.111     0.059    0.438  0.155      0.028      0.050  0.018
19+gdp+unempl_0.1          0.612   0.076  0.027    0.018    3.609e-03  1.276e-03  -0.087   0.027  0.010   -0.123   0.041  0.015           0.084          0.189  0.067     0.223    0.336  0.119      0.031      0.047  0.017
19+gdp+unempl_1.0          0.558   0.114  0.040    0.017    3.558e-03  1.258e-03  -0.086   0.027  0.010   -0.122   0.041  0.014           0.089          0.182  0.064     0.231    0.335  0.119      0.032      0.047  0.017
19+gdp+unempl_5.0          0.477   0.037  0.013    0.014    1.012e-03  3.580e-04  -0.085   0.028  0.010   -0.121   0.044  0.016           0.128          0.171  0.060     0.271    0.305  0.108      0.033      0.047  0.017
19+gdp+unempl_10.0         0.542   0.113  0.040    0.016    3.041e-03  1.075e-03  -0.086   0.030  0.011   -0.121   0.045  0.016           0.132          0.157  0.056     0.274    0.297  0.105      0.032      0.045  0.016
19+gdp+unempl+value_0.1    0.534   0.024  0.009    0.013    4.607e-04  1.629e-04  -0.065   0.013  0.005   -0.097   0.029  0.010           0.393          0.200  0.071     0.485    0.262  0.092      0.052      0.062  0.022
19+gdp+unempl+value_1.0    0.745   0.244  0.086    0.018    5.618e-03  1.986e-03  -0.066   0.015  0.005   -0.097   0.029  0.010           0.387          0.197  0.070     0.481    0.261  0.092      0.051      0.060  0.021
19+gdp+unempl+value_5.0    0.578   0.069  0.024    0.014    1.713e-03  6.058e-04  -0.067   0.015  0.005   -0.097   0.030  0.011           0.399          0.221  0.078     0.488    0.267  0.095      0.051      0.060  0.021
19+gdp+unempl+value_10.0   0.529   0.016  0.006    0.014    1.353e-03  4.782e-04  -0.068   0.017  0.006   -0.098   0.031  0.011           0.385          0.206  0.073     0.478    0.263  0.093      0.050      0.058  0.021